In [33]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

In [34]:
df = pd.read_excel("../data/raw/Reporte SSMOdigital Convenios.xlsx")
df["NomRevisor"] = df["NomRevisor"].str.split().str.join(" ")

## Identificar la procedencia de cada uno de los movimientos de un convenio

En este apartado se quiere identificar la unidad en donde se ha realizado un movimiento a cada
convenio.


En este apartado se calcula la cantidad de tiempo utilizado entre cada uno de los movimiento de
un convenio.


In [35]:
# Deja solamente los convenios y los ordena por su numero y fecha de accion
df_convenios = df.query("Categoria == 'Convenio con Entidades Públicas'").copy()
df_convenios = df_convenios.sort_values(["NumInterno", "FechaHistorico"])

# Calcula el tiempo entre 2 acciones distintas para cada convenio
df_convenios["tiempo_utilizado"] = df_convenios.groupby("NumInterno")["FechaHistorico"].diff()

# Calcula el tiempo total utilizado para cada convenio
tiempo_total_por_convenio = df_convenios.groupby("NumInterno")["FechaHistorico"].apply(
    lambda x: x.max() - x.min()
)

En este apartado se identifica cada una de las personas en el SSMO, y se formatea su nombre para
unirlo a los movimientos de los convenios.


In [36]:
df_personas = pd.read_excel(
    "../data/raw/Plano DSSMO dic 2023.xlsx",
    usecols=[
        "Nombre Funcionario",
        "Código Unidad",
        "Descripción Unidad",
        "Código Unidad 2",
        "Descripción Unidad 2",
    ],
).drop_duplicates()

Los nombres de las personas estan escritos en distintos formatos en las bases de datos.

- En los convenios estan escritos como: PRIMER NOMBRE - PRIMER APELLIDO - SEGUNDO APELLIDO.
- En el registro de personas estan en el formato: PRIMER APELLIDO - SEGUNDO APELLIDO - PRIMER NOMBRE - SEGUNDO NOMBRE.

Por lo tanto, es necesario llegar a un formato similar.


In [37]:
nombres_separados = df_personas["Nombre Funcionario"].str.split()
df_personas["nombre_formateado"] = (
    nombres_separados.str[2] + " " + nombres_separados.str[0] + " " + nombres_separados.str[1]
)
df_personas = df_personas[["nombre_formateado", "Nombre Funcionario", "Descripción Unidad"]]

In [38]:
df_convenios.merge(df_personas, how="left", left_on="NomRevisor", right_on="nombre_formateado")

,Procedencia,TipoDocumento,Categoria,NumInterno,Materia,AccionDoc,EstadoDoc,NomRevisor,FechaHistorico,DocConvAsociado,Unnamed: 10,tiempo_utilizado,nombre_formateado,Nombre Funcionario,Descripción Unidad
0,DEPARTAMENTO DE ASESORÍA JURÍDICA,Contratos,Convenio con Entidades Públicas,1030010-4,CONVENIO ENTRE EL SERVICIO DE SALUD METROPOLIT...,Solicitar Revision,En Revision,GISELLE MELLA GONZALEZ,2023-04-24 12:12:59,NaN,NaN,NaT,GISELLE MELLA GONZALEZ,MELLA GONZALEZ GISELLE ARACELLY,DEPTO ASESORIA JURIDICA DSSMO
1,DEPARTAMENTO DE ASESORÍA JURÍDICA,Contratos,Convenio con Entidades Públicas,1030010-4,CONVENIO ENTRE EL SERVICIO DE SALUD METROPOLIT...,Revisado,En Revision,PAULINA FUENTES DIAZ,2023-04-24 17:26:00,NaN,NaN,0 days 05:13:01,PAULINA FUENTES DIAZ,FUENTES DIAZ PAULINA VERONICA,DEPTO ASESORIA JURIDICA DSSMO
2,DEPARTAMENTO DE ASESORÍA JURÍDICA,Contratos,Convenio con Entidades Públicas,1030010-4,CONVENIO ENTRE EL SERVICIO DE SALUD METROPOLIT...,Rechazar,Rechazado,SILVANA MUÑOZ MUÑOZ,2023-04-26 11:01:28,NaN,NaN,1 days 17:35:28,SILVANA MUÑOZ MUÑOZ,MUÑOZ MUÑOZ SILVANA GERALDINE,DEPTO. DE ABASTECIMIENTO DSSMO
3,DEPARTAMENTO DE ASESORÍA JURÍDICA,Contratos,Convenio con Entidades Públicas,1030010-4,CONVENIO ENTRE EL SERVICIO DE SALUD METROPOLIT...,Solicitar Revision,En Revision,GISELLE MELLA GONZALEZ,2023-04-26 11:08:52,NaN,NaN,0 days 00:07:24,GISELLE MELLA GONZALEZ,MELLA GONZALEZ GISELLE ARACELLY,DEPTO ASESORIA JURIDICA DSSMO
4,DEPARTAMENTO DE ASESORÍA JURÍDICA,Contratos,Convenio con Entidades Públicas,1030010-4,CONVENIO ENTRE EL SERVICIO DE SALUD METROPOLIT...,Revisado,En Revision,PAULINA FUENTES DIAZ,2023-04-27 15:40:45,NaN,NaN,1 days 04:31:53,PAULINA FUENTES DIAZ,FUENTES DIAZ PAULINA VERONICA,DEPTO ASESORIA JURIDICA DSSMO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,DEPARTAMENTO DE CAPACITACIÓN,Contratos,Convenio con Entidades Públicas,4040010-9,CONVENIO PROGRAMA DE CAPACITACION Y FORMACIO...,Solicitar Revision,En Revision,LUZ MARIA BRITO GONZALEZ,2023-05-02 16:17:32,NaN,NaN,NaT,NaN,NaN,NaN
2236,DEPARTAMENTO DE CAPACITACIÓN,Contratos,Convenio con Entidades Públicas,4040010-9,CONVENIO PROGRAMA DE CAPACITACION Y FORMACIO...,Revisado,En Revision,PAULINA FUENTES DIAZ,2023-05-03 11:19:47,NaN,NaN,0 days 19:02:15,PAULINA FUENTES DIAZ,FUENTES DIAZ PAULINA VERONICA,DEPTO ASESORIA JURIDICA DSSMO
2237,DEPARTAMENTO DE CAPACITACIÓN,Contratos,Convenio con Entidades Públicas,4040010-9,CONVENIO PROGRAMA DE CAPACITACION Y FORMACIO...,Revisado,En Revision,JAVIER SARAVIA MORA,2023-05-03 15:06:18,NaN,NaN,0 days 03:46:31,JAVIER SARAVIA MORA,SARAVIA MORA JAVIER IGNACIO,DEPTO CAPACITACION DSSMO
2238,DEPARTAMENTO DE CAPACITACIÓN,Contratos,Convenio con Entidades Públicas,4040010-9,CONVENIO PROGRAMA DE CAPACITACION Y FORMACIO...,Revisado,En Revision,ALVARO SVERLIJ ZAPATA,2023-05-03 15:50:41,NaN,NaN,0 days 00:44:23,ALVARO SVERLIJ ZAPATA,SVERLIJ ZAPATA ALVARO YARY,DEPTO CAPACITACION DSSMO
